# GOOGLENET : train/test/valid set로 나눈 뒤 학습 

In [ ]:
from tensorflow.keras.layers import Concatenate, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import os


def inception_module(x, base_channels=32):
    a = Conv2D(base_channels*2, 1, 1, activation='relu')(x)
    b_1 = Conv2D(base_channels*4, 1, 1, activation='relu')(x)
    b_2 = Conv2D(base_channels*4, 3, 1, padding='same', activation='relu')(b_1)
    c_1 = Conv2D(base_channels, 1, 1, activation='relu')(x)
    c_2 = Conv2D(base_channels, 5, 1, padding='same', activation='relu')(c_1)
    d_1 = MaxPooling2D(3, 1, padding='same')(x)
    d_2 = Conv2D(base_channels, 1, 1, activation='relu')(d_1)
    return Concatenate(axis=-1)([a, b_2, c_2, d_2])


def googleNet_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = Conv2D(64, (7, 7), strides=(2, 2),
               padding='same', activation='relu')(inputs)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = inception_module(x, base_channels=32)
    x = inception_module(x, base_channels=32)
    x = inception_module(x, base_channels=32)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model


data_dir = '/Users/eunjincho/Documents/workspaces/data_geeks/emotion/'
img_width, img_height = 224, 224
num_classes = len(os.listdir(data_dir)) # 상위 디렉토리 내에 있는 감정 폴더의 갯수로 

# 전처리
datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2,

    # 이미지 
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

batch_size = 32

# 훈련셋, 검증셋, 테스트셋으로 데이터 나누고 전처리 
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Create and compile the model
model = googleNet_model((img_width, img_height, 3), num_classes)
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=10
)

# Optionally, evaluate the model on the test set
test_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    # Assuming you have created a 'test' directory within each emotion directory
    subset='validation'
)

loss, accuracy = model.evaluate_generator(test_generator)
print("Test loss:", loss)
print("Test accuracy:", accuracy)